股票系統介紹
===
股票測試網址: https://merry.ee.ncku.edu.tw/stock/  
帳號為luffy帳號或學號，密碼為成功入口密碼

## 介紹
伺服器上有準備從2004年到現在每天台股的股市資料，透過這些原始資料，要實作一個預測器來決定使用者明天要怎麼投資股票，購買和賣出的結果可以由上述的網址上登入後看到詳細行為，而每個人的目前的總資產會依照時間化成圖和其他人比較。
## 股票資料
在luffy的 /home/mlb/res/stock/twse/json 資料夾中，存放著每一天每一支股票的編號、開牌價、收盤價、最高最低價、成交量等資料。  
資料的格式如下範例:

In [4]:
stock =   {
             "0050" : {
                "adj_close" : "84.90",
                "close" : "84.95",
                "high" : "85.80",
                "low" : "84.90",
                "open" : "85.80",
                "volume" : "20996200"
             },
            "0051" : {
               "adj_close" : "32.50",
               "close" : "32.50",
               "high" : "33.20",
               "low" : "32.46",
               "open" : "33.20",
               "volume" : "92075"
            },
            "0052" : {
               "adj_close" : "54.30",
               "close" : "54.50",
               "high" : "54.90",
               "low" : "54.50",
               "open" : "54.90",
               "volume" : "102150"
            }
    }

print(stock['0050'])
print(stock['0050']['close'])

{'close': '84.95', 'volume': '20996200', 'open': '85.80', 'adj_close': '84.90', 'low': '84.90', 'high': '85.80'}
84.95


## 產生預測模型
為了訓練股票預測器，必須從上述的json股票資料中根據自定義的時間區段、股票特徵種類等切分出訓練資料。  
例如: 取預測日當日前一周的開盤價、收盤價以及平均的價格來訓練模型。  
## 產生Decision File
Decision File指的是最終決定要買哪一隻股票、要買多少錢以及要讓這支股票存活多久等條件。Decision File的格式也是json，是以一個array包在最外層，每一支股票用一個object表示。  

範例如下:

In [5]:
[ # 一個陣列，每個元素是一個 object
  {
    "code": "1214", # 股票代碼
    "type": "buy", # 兩種操作，買(buy)/放空(short)
    "weight": 1, # 買幾張股票; 每個人一開始有 1 單位的虛擬貨幣(以這個例子為例，用 1/4 買 1214；用 3/4 放空 2351)
    "life": 5, # 5 天後會賣(其中一個條件)
    "open_price": 16.75,# 等於或小於這個價格買
    "close_high_price": 16.8,# 比這個價格高就賣
    "close_low_price": 16.3 # 停損點
  },
  {
    "code": "2351",
    "life": 3,
    "type": "short",
    "weigth": 3,
    "open_price": 27.3,
    "close_high_price": 27.5,
    "close_low_price": 25.8
  }
]


[{'close_high_price': 16.8,
  'close_low_price': 16.3,
  'code': '1214',
  'life': 5,
  'open_price': 16.75,
  'type': 'buy',
  'weight': 1},
 {'close_high_price': 27.5,
  'close_low_price': 25.8,
  'code': '2351',
  'life': 3,
  'open_price': 27.3,
  'type': 'short',
  'weigth': 3}]

## 股票權重
每個人每一天有一單位的錢可以花，在計算上系統會將所有權重加起來算成百分比。假設只買兩支股票，1214買1個權重，2351買3個權重，那經過計算後花的錢分別是0.25、0.75。
## 買股票的賣出條件
- 今日最高價大於`close high price`
- 今日最低價小於`close low price`
- 價格都介於`close high price`和`close low price`間，但股票已經買進`life`天

## 放空股票的買入條件
- 今日最高價小於`close high price`
- 今日最低價大於`close low price`
- 價格都介於`close high price`和`close low price`間，但股票已經放空`life`天

## 未操作懲罰
如果當日沒有任何股票操作(沒有 decision.json 或是檔案內容是空的)，等同於投資大盤。  
  
投資大盤的decision file如下:

In [9]:
[
  {
    "code": "0050",
    "life": 1,
    "type": "buy",
    "weight": 1,
    "open_price": 'that days opening price',
    "close_high_price": 'that days highest price + 1',
    "close_low_price":' that days lowest price - 1'
  }
]

[{'close_high_price': 'that days highest price + 1',
  'close_low_price': ' that days lowest price - 1',
  'code': '0050',
  'life': 1,
  'open_price': 'that days opening price',
  'type': 'buy',
  'weight': 1}]

## 股票繳交要求